In [2]:
from qiskit import *
from qiskit.quantum_info import SparsePauliOp
from mimiqcircuits import *
import numpy as np

# La Hamiltoniana data
ham = SparsePauliOp.from_list([('IIZZ', 0.5 + 0.j), 
                               ('IZIZ', 0.5 + 0.j), 
                               ('IZZI', 0.5 + 0.j), 
                               ('ZIIZ', 0.5 + 0.j), 
                               ('ZZII', 0.5 + 0.j)])

# Funzione per trasformare in una lista di oggetti PauliString
def pauli_string_converter(ham):
    pauli_objects = []
    for term, coeff in ham.to_list():
        # Crea un PauliString da ciascun termine
        pauli_string = PauliString(term)
        # Estrai i coefficienti e ordina gli indici se necessario
        pauli_objects.append((coeff, pauli_string))
    return pauli_objects

# Traduci l'Hamiltoniana
pauli_objects = pauli_string_converter(ham)

print(pauli_objects)

# Mostra gli oggetti PauliString con i relativi coefficienti
for obj in pauli_objects:
    print(f"Peso: {obj[0]}, PauliString: {obj[1]}")


[((0.5+0j), IIZZ), ((0.5+0j), IZIZ), ((0.5+0j), IZZI), ((0.5+0j), ZIIZ), ((0.5+0j), ZZII)]
Peso: (0.5+0j), PauliString: IIZZ
Peso: (0.5+0j), PauliString: IZIZ
Peso: (0.5+0j), PauliString: IZZI
Peso: (0.5+0j), PauliString: ZIIZ
Peso: (0.5+0j), PauliString: ZZII


/Users/dario/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
conn = MimiqConnection(url="https://mimiq.qperfect.io/api")
#conn.connect()

username = "dmelegari@infn.it"
password = "ThaNgeejoS7e"

conn.connectUser(username, password)


Connection:
├── url: https://mimiq.qperfect.io/api
├── Computing time: 258/3000 minutes
├── Max time limit per request: 180 minutes
├── Default time limit is equal to max time limit: 180 minutes
└── status: open

In [4]:
# Create the ansatz
n_qubits = 4
n_shots = 1024

# Random parameters for the ansatz
theta  = np.random.uniform(0, 2 * np.pi, size=n_qubits)
theta2 = np.random.uniform(0, 2 * np.pi, size=n_qubits)

# Define the circuit
qc = Circuit()

# First layer: Hadamard gates
for i in range(n_qubits):
    qc.push(GateH(), i)

# First layer: RX gates
for i in range(n_qubits):
    qc.push(GateRX(theta[i]), i)

# First layer: CNOTs in a chain
for i in range(n_qubits - 1):
    qc.push(GateCX(), i, i + 1)

# Second layer: RX gates with new parameters
for i in range(n_qubits):
    qc.push(GateRX(theta2[i]), i)

# Second layer: CNOTs in a chain
for i in range(n_qubits - 1):
    qc.push(GateCX(), i, i + 1)

# Draw the circuit
qc.draw()




        ┌─┐         ┌──────────────────────┐                                    
 q[0]: ╶┤H├─────────┤RX(1.1782281858664183)├───────────────────────────────────╴
        └─┘┌─┐      └──────────────────────┘┌─────────────────────┐             
 q[1]: ╶───┤H├──────────────────────────────┤RX(3.619433096642063)├────────────╴
           └─┘┌─┐                           └─────────────────────┘             
 q[2]: ╶──────┤H├──────────────────────────────────────────────────────────────╴
              └─┘┌─┐                                                            
 q[3]: ╶─────────┤H├───────────────────────────────────────────────────────────╴
                 └─┘                                                            
                                                                                
...
                                                                                
 q[0]: ╶────────────────────────────────────────────────●──────────────────────╴
                        

In [5]:
# Function that works as in Qiskit: the idea is that we can only measure in the Z basis, so that:
#
# - If in the Hamiltonian there is a X pauli string --> apply H
# - If in the Hamiltonian there is a Y pauli string --> apply H + SDG
#

def apply_meas_rotation(qc, pauli_string):
    """"
    Apply the correct rotations to measure in the correct basis
    """

    pauli_str = str(pauli_string)  # Converti in stringa se necessario
    for qubit, pauli in enumerate(pauli_str):
        if pauli == 'X':
            qc.push(GateH(), qubit)  # Hadamard per misurare in X
        elif pauli == 'Y':
            qc.push(GateSDG(), qubit)  # S† seguito da Hadamard per misurare in Y
            qc.push(GateH(), qubit)


# Calculate the expectation value of the Hamiltonian over a circuit

def cost_function(circuit, pauli_objects, n_shots):
    
    cost_fun = 0
    
    for coeff, pauli_string in pauli_objects:
        # copy the circuit to avoid undesired modifications
        qc_copy = circuit.copy()
        
        # apply the correct rotations to measure in the correct basis
        apply_meas_rotation(qc_copy, pauli_string)
        
        # add the measure for each qubit (like measure_all of qiskit)
        for i in range(n_qubits):
            qc_copy.push(Measure(), i, i)
        
        # Esegue il circuito
        j_id = conn.execute(qc_copy, nsamples=n_shots)
        
        # Attende il completamento del job
        #while not conn.isJobDone(j_id):
        #    pass
    
        # Ottiene i risultati
        res = conn.get_result(j_id)
                
        # calculate the expectation value
        exp_value = sum(coeff if str(bs).count("1") % 2 == 0 else -coeff for bs in res.cstates) / n_shots
        cost_fun += exp_value

        print(cost_fun)
    
    return np.real(cost_fun)

cost = cost_function(qc, pauli_objects, n_shots)
print("Valore di aspettazione:", cost)


KeyboardInterrupt: 

In [5]:
from scipy.optimize import minimize
import numpy as np

# Wrapper function for the optimizer
def cost_function_opt(theta, pauli_objects, n_shots):
    qc = Circuit()

    # Apply Hadamard gates
    for i in range(n_qubits):
        qc.push(GateH(), i)

    # Apply parameterized RX gates
    for i in range(n_qubits):
        qc.push(GateRX(theta[i]), i)

    return cost_function(qc, pauli_objects, n_shots)

# Initial random parameters
theta_init = np.random.uniform(0, 2 * np.pi, size=n_qubits)

# Run optimization using COBYLA
result = minimize(cost_function_opt, theta_init, args=(pauli_objects, n_shots), method='COBYLA')

# Extract optimal parameters and cost
optimal_theta = result.x
optimal_cost = result.fun

print("Optimal parameters:", optimal_theta)
print("Minimum cost function value:", optimal_cost)



(-0.021484375+0j)
(-0.01953125+0j)
(-0.02734375+0j)
(-0.0361328125+0j)
(-0.0546875+0j)
(-0.0029296875+0j)
(-0.0029296875+0j)
(-0.001953125+0j)
(-0.0263671875+0j)
(-0.001953125+0j)
(-0.017578125+0j)
(-0.017578125+0j)
(-0.0263671875+0j)
(-0.0078125+0j)
(-0.01953125+0j)
(0.03125+0j)
(0.0302734375+0j)
(0.009765625+0j)
0j
(-0.0546875+0j)
(-0.0234375+0j)
(-0.033203125+0j)
(-0.0654296875+0j)
(-0.0869140625+0j)
(-0.0830078125+0j)
(0.015625+0j)
(-0.005859375+0j)
(0.0146484375+0j)
(0.0380859375+0j)
(0.048828125+0j)
(-0.00390625+0j)
(-0.015625+0j)
(-0.0263671875+0j)
(-0.033203125+0j)


Traceback (most recent call last):
capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
  File "/Users/dario/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_cobyla_py.py", line 281, in calcfc
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
    f = sf.fun(x)
  File "/Users/dario/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_differentiable_functions.py", line 278, in fun
    self._update_fun()
  File "/Users/dario/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_differentiable_functions.py", line 262, in _update_fun
    self._update_fun_impl()
  File "/Users/dario/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_differentiable_functions.py", line 163, in update_fun
    self.f = fun_wrapped(self.x)
  File "/Users/dario/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_differentiable_functions.py", line 145, in fun_wra

: 